#### Imports

In [65]:
import pandas as pd
import datetime
import numpy as np
import math
import re

#### Load Data

In [66]:
PATH = 'data/'
file_name = 'KPMG_VI_New_raw_data_update_final.xlsx'

In [67]:
transaction = pd.read_excel(PATH + file_name, sheet_name='Transactions', header=1, parse_dates = ['product_first_sold_date','transaction_date'])
cust_demo = pd.read_excel(PATH + file_name, sheet_name='CustomerDemographic', header=1)
cust_add = pd.read_excel(PATH + file_name, sheet_name='CustomerAddress', header=1)
new_cust = pd.read_excel(PATH + file_name, sheet_name='NewCustomerList', header=1)



/var/folders/k2/zvczrjsj2wn7bpkxmhrtnt5r0000gn/T/ipykernel_73441/3237879611.py:2: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  cust_demo = pd.read_excel(PATH + file_name, sheet_name='CustomerDemographic', header=1)
/var/folders/k2/zvczrjsj2wn7bpkxmhrtnt5r0000gn/T/ipykernel_73441/3237879611.py:4: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  new_cust = pd.read_excel(PATH + file_name, sheet_name='NewCustomerList', header=1)


#### Transactions Data Quality Check

In [68]:
print(transaction.head())

   transaction_id  product_id  customer_id transaction_date  online_order  \
0               1           2         2950       2017-02-25           0.0   
1               2           3         3120       2017-05-21           1.0   
2               3          37          402       2017-10-16           0.0   
3               4          88         3135       2017-08-31           0.0   
4               5          78          787       2017-10-01           1.0   

  order_status           brand product_line product_class product_size  \
0     Approved           Solex     Standard        medium       medium   
1     Approved   Trek Bicycles     Standard        medium        large   
2     Approved      OHM Cycles     Standard           low       medium   
3     Approved  Norco Bicycles     Standard        medium       medium   
4     Approved  Giant Bicycles     Standard        medium        large   

   list_price  standard_cost product_first_sold_date  
0       71.49          53.62         

In [69]:
transaction.describe()

,transaction_id,product_id,customer_id,online_order,list_price,standard_cost
count,20000.000000,20000.00000,20000.000000,19640.000000,20000.000000,19803.000000
mean,10000.500000,45.36465,1738.246050,0.500458,1107.829449,556.046951
std,5773.647028,30.75359,1011.951046,0.500013,582.825242,405.955660
min,1.000000,0.00000,1.000000,0.000000,12.010000,7.210000
25%,5000.750000,18.00000,857.750000,0.000000,575.270000,215.140000
50%,10000.500000,44.00000,1736.000000,1.000000,1163.890000,507.580000
75%,15000.250000,72.00000,2613.000000,1.000000,1635.300000,795.100000
max,20000.000000,100.00000,5034.000000,1.000000,2091.470000,1759.850000


In [70]:
missing_values = transaction.isnull().sum()
print(f"Number of missing values:\n{missing_values}")

Number of missing values:
transaction_id               0
product_id                   0
customer_id                  0
transaction_date             0
online_order               360
order_status                 0
brand                      197
product_line               197
product_class              197
product_size               197
list_price                   0
standard_cost              197
product_first_sold_date    197
dtype: int64


In [71]:
# looking at the percentage of data that's missing 
empty_records = round((197/20000)*100,2)
print('Percentage of empty records ', math.ceil(empty_records),'%')

Percentage of empty records  1 %


In [72]:
# Check for duplicate records
duplicates = transaction.duplicated()
print(f"Number of duplicate records: {duplicates.sum()}")

Number of duplicate records: 0


In [73]:
# looking at unique customers 
trans_cust_unique = len(transaction.customer_id.unique())
print('unique customers : ', trans_cust_unique)

unique customers :  3494


In [74]:
# checking data types 
transaction.dtypes

transaction_id                      int64
product_id                          int64
customer_id                         int64
transaction_date           datetime64[ns]
online_order                      float64
order_status                       object
brand                              object
product_line                       object
product_class                      object
product_size                       object
list_price                        float64
standard_cost                     float64
product_first_sold_date            object
dtype: object

In [75]:
# Convert transaction_date and product_first_sold_date to datetime objects

# converts an Excel serial date (represented as a float) to a datetime object and then formats it as a string in the format '%Y-%m-%d'
def date_conv(date):
    if pd.isnull(date):
        return None

    # Define the reference date (Excel epoch)
    epoch = datetime.datetime(1899, 12, 30)

    # Convert the serial date value to a datetime object
    serial_date = int(date)
    date = epoch + datetime.timedelta(days=serial_date)

    # Format the date as a string in the desired format
    formatted_date = date.strftime('%Y-%m-%d')

    # Print the formatted date
    return formatted_date

transaction['product_first_sold_date'] = transaction['product_first_sold_date'].apply(date_conv)

In [76]:
transaction.sort_values('product_first_sold_date').head()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
13882,13883,98,725,2017-03-14,0.0,Approved,OHM Cycles,Standard,medium,medium,795.34,101.58,1991-01-21
13411,13412,57,1531,2017-03-11,1.0,Approved,WeareA2B,Touring,medium,large,1890.39,260.14,1991-01-21
1548,1549,0,240,2017-09-14,0.0,Approved,OHM Cycles,Standard,medium,medium,183.86,137.90,1991-01-21
11494,11495,56,2703,2017-09-18,0.0,Approved,OHM Cycles,Standard,medium,medium,183.86,137.90,1991-01-21
10036,10037,57,235,2017-07-27,0.0,Approved,WeareA2B,Touring,medium,large,1890.39,260.14,1991-01-21


It seems that the first product was sold in 1991, making it 32 years old. That could be due to an entry error as it's not likely.

In [77]:
# as per business requirements, data needs to be 3 months old 
transaction['transaction_date'].agg(['min', 'max'])

min   2017-01-01
max   2017-12-30
Name: transaction_date, dtype: datetime64[ns]

The data encompasses a time span of 12 months instead of 3 months. 

In [78]:
# Check unique values in categorical columns

print('order_status', transaction.order_status.unique())
print('brand', transaction.brand.unique())
print('product_line', transaction.product_line.unique())
print('product_class', transaction.product_class.unique())
print('product_size', transaction.product_size.unique())


order_status ['Approved' 'Cancelled']
brand ['Solex' 'Trek Bicycles' 'OHM Cycles' 'Norco Bicycles' 'Giant Bicycles'
 'WeareA2B' nan]
product_line ['Standard' 'Road' 'Mountain' 'Touring' nan]
product_class ['medium' 'low' 'high' nan]
product_size ['medium' 'large' 'small' nan]


In [79]:
# adding revenue column
transaction['profit'] = transaction['list_price'] - transaction['standard_cost']

In [80]:
transaction['profit'].describe()

count    19803.000000
mean       551.950915
std        493.412849
min          4.800000
25%        133.780000
50%        445.210000
75%        830.240000
max       1702.550000
Name: profit, dtype: float64

In [81]:
# since 1% of the data is missing, identify which product it belongs to
condition =  (transaction['brand'].isna()) & (transaction['product_line'].isna()) & (transaction['product_class'].isna()) & (transaction['standard_cost'].isna()) 
missing_data = transaction[condition]


product_missing_data = missing_data.product_id.unique()

print('Missing data belongs to these products: ', product_missing_data)

Missing data belongs to these products:  [0]


In [82]:
transaction.head()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date,profit
0,1,2,2950,2017-02-25,0.0,Approved,Solex,Standard,medium,medium,71.49,53.62,2012-12-02,17.87
1,2,3,3120,2017-05-21,1.0,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,2014-03-03,1702.55
2,3,37,402,2017-10-16,0.0,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,1999-07-20,1544.61
3,4,88,3135,2017-08-31,0.0,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10,1998-12-16,817.36
4,5,78,787,2017-10-01,1.0,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48,2015-08-10,1055.82


In [83]:
# save to csv
transaction.to_csv(PATH + 'transaction.csv')

#### Customer Demographic Quality Check

In [84]:
print(cust_demo.head())

   customer_id      first_name  last_name  gender  \
0            1         Laraine  Medendorp       F   
1            2             Eli    Bockman    Male   
2            3           Arlin     Dearle    Male   
3            4          Talbot        NaN    Male   
4            5  Sheila-kathryn     Calton  Female   

   past_3_years_bike_related_purchases        DOB               job_title  \
0                                   93 1953-10-12     Executive Secretary   
1                                   81 1980-12-16  Administrative Officer   
2                                   61 1954-01-20      Recruiting Manager   
3                                   33 1961-10-03                     NaN   
4                                   56 1977-05-13           Senior Editor   

  job_industry_category     wealth_segment deceased_indicator  \
0                Health      Mass Customer                  N   
1    Financial Services      Mass Customer                  N   
2              Property

Transaction table and Customer demographic table seem to be joined on customer ID 

In [85]:
cust_demo.describe()

,customer_id,past_3_years_bike_related_purchases,tenure
count,4000.000000,4000.000000,3913.000000
mean,2000.500000,48.890000,10.657041
std,1154.844867,28.715005,5.660146
min,1.000000,0.000000,1.000000
25%,1000.750000,24.000000,6.000000
50%,2000.500000,48.000000,11.000000
75%,3000.250000,73.000000,15.000000
max,4000.000000,99.000000,22.000000


In [86]:
missing_values_demo = cust_demo.isnull().sum()
print(f"Number of missing values:\n{missing_values_demo}")



Number of missing values:
customer_id                              0
first_name                               0
last_name                              125
gender                                   0
past_3_years_bike_related_purchases      0
DOB                                     87
job_title                              506
job_industry_category                  656
wealth_segment                           0
deceased_indicator                       0
default                                302
owns_car                                 0
tenure                                  87
dtype: int64


Some customers don't have a last name and/or dob, job_title,job_industry_category

In [87]:
# Check for duplicate records
duplicates_cust_demo = cust_demo.duplicated()
print(f"Number of duplicate records: {duplicates_cust_demo.sum()}")


Number of duplicate records: 0


In [88]:
demo_cust_unique = len(cust_demo.customer_id.unique())
print('unique customers in transaction data set {} \nunique customers in customer demographic dataset {}'.format(trans_cust_unique, demo_cust_unique))


unique customers in transaction data set 3494 
unique customers in customer demographic dataset 4000


There is some discrepancy in the customer data 

In [89]:
# exploration data types 
print(cust_demo.dtypes)

customer_id                                     int64
first_name                                     object
last_name                                      object
gender                                         object
past_3_years_bike_related_purchases             int64
DOB                                    datetime64[ns]
job_title                                      object
job_industry_category                          object
wealth_segment                                 object
deceased_indicator                             object
default                                        object
owns_car                                       object
tenure                                        float64
dtype: object


In [90]:
print('gender' , cust_demo.gender.unique())
print('job_industry_category' , cust_demo.job_industry_category.unique())
print('wealth_segment' , cust_demo.wealth_segment.unique())
print('deceased_indicator' , cust_demo.deceased_indicator.unique())
print('owns_car' , cust_demo.owns_car.unique())
print('tenure' , cust_demo.tenure.unique())

gender ['F' 'Male' 'Female' 'U' 'Femal' 'M']
job_industry_category ['Health' 'Financial Services' 'Property' 'IT' nan 'Retail' 'Argiculture'
 'Manufacturing' 'Telecommunications' 'Entertainment']
wealth_segment ['Mass Customer' 'Affluent Customer' 'High Net Worth']
deceased_indicator ['N' 'Y']
owns_car ['Yes' 'No']
tenure [11. 16. 15.  7.  8. 13. 20.  9.  6.  1. 18. 21. 12. 19. 14.  4. 22.  5.
 17.  2.  3. 10. nan]


In some cases Yes adn No, is "Yes" & "No" and in others "Y" , "N". And female is "F" or "Female" or "Femal".  It's best to standardize these values

In [91]:
# handling gender 
# standardizing gender
def standardize_gender(gender):
    if re.search(r'^(Femal|Femal(e)?)$', gender, re.IGNORECASE):
        return 'Female'
    elif re.search('^F', gender):
        return 'Female'
    elif re.search(r'^(M)$', gender, re.IGNORECASE):
        return 'Male'
    elif re.search(r'^(U)$', gender, re.IGNORECASE):
        return 'Undefined'
    else:
        return gender
    

cust_demo['gender']  = cust_demo['gender'].apply(standardize_gender)

print("gender types : ", cust_demo['gender'].value_counts())

gender types :  Female       2039
Male         1873
Undefined      88
Name: gender, dtype: int64


In [92]:
# converting deceased_indicator to Yes & No for consistency 
cust_demo['deceased_indicator'] = cust_demo['deceased_indicator'].replace({'N':'No', 'Y': 'Yes'})

print('deceased_indicator values ', cust_demo['deceased_indicator'].unique())

deceased_indicator values  ['No' 'Yes']


In [93]:
# exploring default column
print('default' , cust_demo.default.unique()[:6])

default ['"\'' "<script>alert('hi')</script>" datetime.datetime(2018, 2, 1, 0, 0)
 '() { _; } >_[$($())] { touch /tmp/blns.shellshock2.fail; }' 'NIL'
 'ðµ ð ð ð']


Seems to have corrupted text, perhaps was a non-UTF character set that was improperly handled or converted

In [94]:
# adding age column, could be helpful for the remainder of the course 
cust_demo['age'] =  2023 - cust_demo['DOB'].dt.year

In [95]:
cust_demo.head()

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,default,owns_car,tenure,age
0,1,Laraine,Medendorp,Female,93,1953-10-12,Executive Secretary,Health,Mass Customer,No,"""'",Yes,11.0,70.0
1,2,Eli,Bockman,Male,81,1980-12-16,Administrative Officer,Financial Services,Mass Customer,No,<script>alert('hi')</script>,Yes,16.0,43.0
2,3,Arlin,Dearle,Male,61,1954-01-20,Recruiting Manager,Property,Mass Customer,No,2018-02-01 00:00:00,Yes,15.0,69.0
3,4,Talbot,NaN,Male,33,1961-10-03,NaN,IT,Mass Customer,No,() { _; } >_[$($())] { touch /tmp/blns.shellsh...,No,7.0,62.0
4,5,Sheila-kathryn,Calton,Female,56,1977-05-13,Senior Editor,NaN,Affluent Customer,No,NIL,Yes,8.0,46.0


In [96]:
cust_demo.to_csv(PATH + 'customer_demographic.csv')

#### Customer Address Quality Check

In [97]:
cust_add.head()

,customer_id,address,postcode,state,country,property_valuation
0,1,060 Morning Avenue,2016,New South Wales,Australia,10
1,2,6 Meadow Vale Court,2153,New South Wales,Australia,10
2,4,0 Holy Cross Court,4211,QLD,Australia,9
3,5,17979 Del Mar Point,2448,New South Wales,Australia,4
4,6,9 Oakridge Court,3216,VIC,Australia,9


Transaction table and Customer demographic table and Customer Address seem to be joined on customer ID  

In [98]:
cust_add.describe()

,customer_id,postcode,property_valuation
count,3999.000000,3999.000000,3999.000000
mean,2003.987997,2985.755939,7.514379
std,1154.576912,844.878364,2.824663
min,1.000000,2000.000000,1.000000
25%,1004.500000,2200.000000,6.000000
50%,2004.000000,2768.000000,8.000000
75%,3003.500000,3750.000000,10.000000
max,4003.000000,4883.000000,12.000000


In [99]:
missing_values_add = cust_add.isnull().sum()
print(f"Number of missing values:\n{missing_values_add}")


Number of missing values:
customer_id           0
address               0
postcode              0
state                 0
country               0
property_valuation    0
dtype: int64


In [100]:
# Check for duplicate records
duplicates_cust_add = cust_add.duplicated()
print('duplicated rows', duplicates_cust_add.sum())


duplicated rows 0


In [101]:
add_cust_unique = len(cust_add.customer_id.unique())

print('unique customers in transaction data set {} \nunique customers in customer demographic dataset {} \
\nunique customers in customer address dataset {}'.format(trans_cust_unique, demo_cust_unique, add_cust_unique))

unique customers in transaction data set 3494 
unique customers in customer demographic dataset 4000 
unique customers in customer address dataset 3999


In [102]:
cust_add.dtypes

customer_id            int64
address               object
postcode               int64
state                 object
country               object
property_valuation     int64
dtype: object

In [103]:
# checking the postcode 
cond = cust_add['postcode'].apply(lambda x: len(str(x)) > 4)
invalid_postcodes = cust_add[cond]

print('Number of invalid postcodes : ', len(invalid_postcodes))

Number of invalid postcodes :  0


In [104]:
print('state' , cust_add.state.unique())
print('country' , cust_add.country.unique())

state ['New South Wales' 'QLD' 'VIC' 'NSW' 'Victoria']
country ['Australia']


The state column has some abbreviations and some whole states written, like QLQ, and Victoria

In [105]:
# standardizing state 
mapping = {
    'New South Wales': 'NSW',
    'Victoria': 'VIC'
}

def map_states(state):
    if state in mapping.keys():
        return mapping[state]
    else:
        return state

cust_add['state'] = cust_add['state'].apply(map_states)

In [106]:
cust_add.head()

,customer_id,address,postcode,state,country,property_valuation
0,1,060 Morning Avenue,2016,NSW,Australia,10
1,2,6 Meadow Vale Court,2153,NSW,Australia,10
2,4,0 Holy Cross Court,4211,QLD,Australia,9
3,5,17979 Del Mar Point,2448,NSW,Australia,4
4,6,9 Oakridge Court,3216,VIC,Australia,9


In [107]:
cust_add.to_csv(PATH + 'customer_address.csv')

#### New Customer List Quality Check

In [108]:
new_cust.head()

,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,...,state,country,property_valuation,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Rank,Value
0,Chickie,Brister,Male,86,1957-07-12,General Manager,Manufacturing,Mass Customer,N,Yes,...,QLD,Australia,6,0.56,0.7000,0.8750,0.743750,1,1,1.718750
1,Morly,Genery,Male,69,1970-03-22,Structural Engineer,Property,Mass Customer,N,No,...,NSW,Australia,11,0.89,0.8900,1.1125,0.945625,1,1,1.718750
2,Ardelis,Forrester,Female,10,1974-08-28,Senior Cost Accountant,Financial Services,Affluent Customer,N,No,...,VIC,Australia,5,1.01,1.0100,1.0100,1.010000,1,1,1.718750
3,Lucine,Stutt,Female,64,1979-01-28,Account Representative III,Manufacturing,Affluent Customer,N,Yes,...,QLD,Australia,1,0.87,1.0875,1.0875,1.087500,4,4,1.703125
4,Melinda,Hadlee,Female,34,1965-09-21,Financial Analyst,Financial Services,Affluent Customer,N,No,...,NSW,Australia,9,0.52,0.5200,0.6500,0.650000,4,4,1.703125


for the unnamed columns check the csv to check if they are relevant
- Unnamed: 16 = randomly generated number 
- Unnamed: 17 = checks if a customer has a car, and multiplies it with the randomly generated number and another number 
- Unnamed: 18 = checks if a customer has a property_valuation>6 and past_3_years_bike_related_purchases<80, multiplies it with a generated number 
- Unnamed: 19 = multiplies Unnamed: 18 with those who have a wealth_segment	 of Mass Customer
- Unnamed: 20 = perfroms a ranking function 


In [109]:
# drop misread columns
new_cust.drop(['Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20'], axis=1, inplace=True)

In [110]:

missing_values_new = new_cust.isnull().sum()
print(f"Number of missing values:\n{missing_values_new}")

Number of missing values:
first_name                               0
last_name                               29
gender                                   0
past_3_years_bike_related_purchases      0
DOB                                     17
job_title                              106
job_industry_category                  165
wealth_segment                           0
deceased_indicator                       0
owns_car                                 0
tenure                                   0
address                                  0
postcode                                 0
state                                    0
country                                  0
property_valuation                       0
Rank                                     0
Value                                    0
dtype: int64


In [111]:
# Check for duplicate records
duplicates_cust_new = new_cust.duplicated()
print('duplicated rows', duplicates_cust_new.sum())


duplicated rows 0


In [112]:
new_cust.dtypes

first_name                                     object
last_name                                      object
gender                                         object
past_3_years_bike_related_purchases             int64
DOB                                    datetime64[ns]
job_title                                      object
job_industry_category                          object
wealth_segment                                 object
deceased_indicator                             object
owns_car                                       object
tenure                                          int64
address                                        object
postcode                                        int64
state                                          object
country                                        object
property_valuation                              int64
Rank                                            int64
Value                                         float64
dtype: object

In [113]:
print(new_cust.gender.unique())
print(new_cust.wealth_segment.unique())
print(new_cust.deceased_indicator.unique())
print(new_cust.owns_car.unique())
print(new_cust.tenure.unique())
print(new_cust.state.unique())
print(new_cust.country.unique())
print(new_cust.property_valuation.unique())

['Male' 'Female' 'U']
['Mass Customer' 'Affluent Customer' 'High Net Worth']
['N']
['Yes' 'No']
[14 16 10  5 19 22  8 17  3  9  4 11 12 13  7 20 15  6 18 21  2  1  0]
['QLD' 'NSW' 'VIC']
['Australia']
[ 6 11  5  1  9  7 10  8  4  2 12  3]


In [114]:
# standardizing yes and no values 
new_cust['deceased_indicator'] = new_cust['deceased_indicator'].replace({'N':'No'})
print('Deceased indicator values :', new_cust['deceased_indicator'].unique())

Deceased indicator values : ['No']


In [115]:
# checking the postcode 
cond = new_cust['postcode'].apply(lambda x: len(str(x)) > 4)
invalid_postcodes = new_cust[cond]

print('Number of invalid postcodes : ', len(invalid_postcodes))

Number of invalid postcodes :  0


In [116]:
# adding age column 
new_cust['age'] = 2023 - new_cust['DOB'].dt.year

In [117]:
new_cust.head()

,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,address,postcode,state,country,property_valuation,Rank,Value,age
0,Chickie,Brister,Male,86,1957-07-12,General Manager,Manufacturing,Mass Customer,No,Yes,14,45 Shopko Center,4500,QLD,Australia,6,1,1.718750,66.0
1,Morly,Genery,Male,69,1970-03-22,Structural Engineer,Property,Mass Customer,No,No,16,14 Mccormick Park,2113,NSW,Australia,11,1,1.718750,53.0
2,Ardelis,Forrester,Female,10,1974-08-28,Senior Cost Accountant,Financial Services,Affluent Customer,No,No,10,5 Colorado Crossing,3505,VIC,Australia,5,1,1.718750,49.0
3,Lucine,Stutt,Female,64,1979-01-28,Account Representative III,Manufacturing,Affluent Customer,No,Yes,5,207 Annamark Plaza,4814,QLD,Australia,1,4,1.703125,44.0
4,Melinda,Hadlee,Female,34,1965-09-21,Financial Analyst,Financial Services,Affluent Customer,No,No,19,115 Montana Place,2093,NSW,Australia,9,4,1.703125,58.0


In [118]:
new_cust.to_csv(PATH + 'new_customers.csv')

</br>

#### Recommendation Report 
Based on the observations made we recommend the following:

<b>Transaction table</b>

<li> Address the missing values in the table by imputing them with the appropriate values or removing them if they cannot be accurately filled.</li>
<li> Investigate and correct the incorrect product_first_sold_date data, as it seems to be an entry error.</li>
<li> Adjust the timeframe of the data to reflect the intended period of 3 months.</li>
<li> Investigate why most of the columns for product_id 0 are null, as this may affect future analyses.</li>
<li> Ensure standardization of values for categorical variables such as brand, product_line, product_class, order_status to avoid confusion.</li>
<li> Investigate why 1% of the data is missing </li>

<b>Customer Demographic table</b>


<li>Ensure that all customers have complete demographic data, including last name, date of birth, job_title, and job_industry_category.</li>
<li>Standardize values for categorical variables such as gender and job_industry_category to ensure consistency.</li>
<li>Investigate the corrupted text to identify and correct any errors in the data.</li>

<b>Customer Address table</b>

<li>Investigate why there is one missing customer from the address table when compared to the customer demographic table.</li>
<li>Standardize values for the state column to ensure consistency.</li>
<li>Implement quality control measures to ensure that all postcodes are valid.</li>
